# Disease Detection from CT Scans by 3D CNN

In [1]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling3D, Input, ZeroPadding3D, BatchNormalization, MaxPooling3D, Concatenate, AveragePooling3D
from keras.layers.core import Dense, Activation, Dropout, Lambda
from keras.layers.convolutional import Conv3D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import multi_gpu_model
import tensorflow as tf

/sw/lsa/centos7/python-anaconda-arc-connect/created-20170421/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from importlib import reload
from sklearn.model_selection import train_test_split

In [3]:
#path = '/nfs/turbo/intmed-bnallamo-turbo/wsliu/Data/CT_scans/'
path = '/nfs/turbo/umms-awaljee/wsliu/Data/CT_scans/'
model_path = path + 'models/'
if not os.path.exists(model_path): 
    os.mkdir(model_path)

In [4]:
batch_size = 1
G = 4

## Prepare Data Generators

In [5]:
df = pd.read_excel(path+'K23_Crohn_RadiologyReport_Labels_27MAR2018.xlsx')

df = df.dropna(subset=['StudyID'])

df['filename'] = ['s'+str(int(f)) for f in df['StudyID']]

df = df.drop_duplicates(subset=['filename'])

In [8]:
df.columns

Index(['Database_ID', 'StudyID', 'Ileum', 'ascending', 'transverse',
       'Descending', 'Sigmoid', 'rectum', 'prox_small_bowel', 'Prox_Ileum',
       'Jejunum', 'Duodenum', 'Stomach', 'Esophagus', 'fistula_enteral',
       'fistula_perianal', 'abscess_enteral', 'abscess_perianal',
       'StrictureNoted', 'Bowel_Dilation_Noted', 'Lumen_Narrowing_Noted',
       'Prior_Surgery', 'Ostomy', 'Pouch', 'CD_Active_AnyLocation',
       'CD_Active_SmallBowel', 'CD_Active_RtColon', 'CD_Active_LtColon',
       'CD_Active_Pancolitis', 'Fistula_Any', 'Abscess_any',
       'Stricture_Suspected', 'Surgery_Prior_Any', 'filename'],
      dtype='object')

In [16]:
df['CD_Active_AnyLocation'].value_counts(dropna=False)

1    2127
0    1353
Name: CD_Active_AnyLocation, dtype: int64

In [6]:
trn_df, tst_df = train_test_split(df, test_size=0.2, stratify=df[['CD_Active_AnyLocation', 'Fistula_Any', 'Abscess_any']])

In [25]:
trn_df['Abscess_any'].value_counts(dropna=False)

0    2639
1     142
Name: Abscess_any, dtype: int64

In [23]:
tst_df['Abscess_any'].value_counts()

0    660
1     36
Name: Abscess_any, dtype: int64

In [7]:
import keras_addon
reload(keras_addon)
from keras_addon import ImageFrameGenerator

In [8]:
gen = ImageFrameGenerator()

In [9]:
trn_gen = gen.flow_from_frame(path+'ndarray/', trn_df, 'filename', ['CD_Active_AnyLocation', 'Fistula_Any', 'Abscess_any'], 
                             target_size=(256, 256, 192), color_mode='3d', batch_size=batch_size*G)

Found 3474 images in the directory.
Using 2779 images to generate mini-batches.


In [10]:
tst_gen = gen.flow_from_frame(path+'ndarray/', tst_df, 'filename', ['CD_Active_AnyLocation', 'Fistula_Any', 'Abscess_any'], 
                             target_size=(256, 256, 192), color_mode='3d', batch_size=batch_size*G)

Found 3474 images in the directory.
Using 695 images to generate mini-batches.


## Model Building

In [11]:
from dense3dnet import Dense3DNet

In [12]:
blocks = [6, 12, 24, 16]

In [13]:
model = Dense3DNet(blocks, growth_rate=12)

In [14]:
with tf.device('/cpu:0'):
    base_model = Dense3DNet(blocks, pooling='avg')

    x = base_model.output
    output_CD = Dense(1, activation='sigmoid', name='CD_Active_AnyLocation')(x)
    output_fist = Dense(1, activation='sigmoid', name='Fistula_Any')(x)
    output_absc = Dense(1, activation='sigmoid', name='Abscess_any')(x)

    model = Model(inputs=base_model.input, outputs=[output_CD, output_fist, output_absc])

In [15]:
parallel_model = multi_gpu_model(model, gpus=G)
parallel_model.compile(optimizer='adam', loss={'CD_Active_AnyLocation':'binary_crossentropy', 'Fistula_Any':'binary_crossentropy', 
                                     'Abscess_any':'binary_crossentropy'}, metrics=['accuracy'], 
             loss_weights={'CD_Active_AnyLocation':0.4, 'Fistula_Any':0.3, 'Abscess_any':0.3})

In [ ]:
hist = parallel_model.fit_generator(trn_gen, steps_per_epoch=trn_gen.n // (batch_size*G), epochs=3, validation_data=tst_gen, 
                    validation_steps=tst_gen.n // (batch_size*G))

Epoch 1/3
568/694 [=======================>......] - ETA: 4:17 - loss: 0.4663 - CD_Active_AnyLocation_loss: 0.6707 - Fistula_Any_loss: 0.4229 - Abscess_any_loss: 0.2371 - CD_Active_AnyLocation_acc: 0.6083 - Fistula_Any_acc: 0.8614 - Abscess_any_acc: 0.9481